In [ ]:
# Installing Dependencies

!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 369.8 kB/s eta 0:00:00


In [ ]:
# Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading libraries

import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.llms import OpenAI
import pandas as pd
import numpy as np
import random
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import re

In [ ]:
RANDOM_SEED = 16

In [ ]:
from secret_key import openapi_key

In [ ]:
# Setting openAI key

from secret_key import openapi_key

os.environ['OPENAI_API_KEY'] = openapi_key


In [ ]:
# Loading the Dataset

df = pd.read_csv("/content/drive/MyDrive/llm/data/Expanded_PROMISE.csv")

In [ ]:
# Checking the loaded dataset

df.head(5)

,Id,RequirementText,Class,Binary_Label
0,1,The product shall be available during normal b...,A,1
1,2,The product shall be available for use 24 hour...,A,1


In [ ]:
# Dataset metadata values

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Id               969 non-null    int64 
 1   RequirementText  969 non-null    object
 2   Class            969 non-null    object
 3   Binary_Label     969 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 30.4+ KB


In [ ]:
# Creating the training and validation split

df_train, df_val= train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
df_train.head()

,Id,RequirementText,Class,Binary_Label
337,33,The system shall allow user to choose their pr...,F,0
528,10,The product shall display each type of ship in...,LF,1
591,4,The Disputes applications shall interface with...,O,1
427,46,The system shall display only 10 matching resu...,F,0
351,38,Can give feedback to the 24X7 Customer Care Se...,F,0


In [ ]:

def gen_data(df,flag = True):

    examples = []

    n = len(df) if flag == True else 6

    for i in range(0,n):

        obj = {
            "requirements" : list(df["RequirementText"])[i],
            "categories" : "Functional" if list(df["Binary_Label"])[i] == 0 else "Non Functional"
        }

        examples.append(obj)


    random.shuffle(examples)

    return examples



In [ ]:
examples = gen_data(df_train,False)

In [ ]:
examples

[{'requirements': 'The Disputes applications shall interface with the Merchant Information Database. The Merchant Information Database provides detailed information with regard to the merchant. All merchant detail information shall be obtained from the Merchant Information Database.',
  'categories': 'Non Functional'},
 {'requirements': 'The product shall display each type of ship in a grid using an image of the specific type of ship.',
  'categories': 'Non Functional'},
 {'requirements': 'The users should be able to easily use the system to successfully complete their budgets  within the stipulated time for completion. With a week of training prior to product implementation  98% of the users shall complete their budgets within the stipulated time.',
  'categories': 'Non Functional'},
 {'requirements': 'Can give feedback to the 24X7 Customer Care Service center about their impression for the site and services.',
  'categories': 'Functional'},
 {'requirements': 'The system shall display

In [ ]:
# prompt_template = '''
# Text: {text}
# {class}
# '''

# example_prompt = PromptTemplate(input_variables=["text","class"],template = prompt_template)

# print(example_prompt.format(**examples[0]))


prompt_template2 = '''
Requirements: {requirements}
Categories: {categories}
'''

example_prompt2 = PromptTemplate(input_variables=["requirements","categories"],template = prompt_template2)

print(example_prompt2.format(**examples[0]))


Requirements: The users should be able to easily use the system to successfully complete their budgets  within the stipulated time for completion. With a week of training prior to product implementation  98% of the users shall complete their budgets within the stipulated time.
Categories: Non Functional



In [ ]:



final_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= "Text: {text}/n",
    input_variables=["text"],
    prefix= 'Classify the sentiment of the following text as "Funtional" or "Non Funtional"'
)



In [ ]:
print(final_prompt.format(text =examples[3]["text"] ))

Classify the sentiment of the following text as "Funtional" or "Non Funtional"


Text: Recreation of Inventory Quantity Adjustment for the period of previous 30 days shall take no more than 30min.
Non Functional



Text: The estimator shall apply selected recycled parts to the collision estimate.
Functional



Text: Velkat staff will be available for phone support between 7am-7pm  M-F to provide technical support to the Izogn administrators.
Non Functional



Text: When the hit results in sinking a ship  the product shall mark the ship as sunk on the defensive grid of the defensive player.
Functional



Text: Unauthorised users can only read data without modifying anything.
Non Functional



Text: Each account must have a separate identifying account number.
Functional


Text: When the hit results in sinking a ship  the product shall mark the ship as sunk on the defensive grid of the defensive player./n


In [ ]:
llm = OpenAI(temperature=0.2)

In [ ]:
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.2, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [ ]:
extraction_chain = LLMChain(llm = llm , prompt = final_prompt , output_key= "class" )

# Prediction

# Using few shot learning with my default prompt templete

In [ ]:
val = gen_data(df_val)

In [ ]:
val

[{'requirements': 'The product shall have a consistent color scheme and fonts.',
  'categories': 'Non Functional'},
 {'requirements': 'All updates to data files or database must be initiated from the Disputes System.',
  'categories': 'Non Functional'},
 {'requirements': 'Website must be able to support free trial periods with various parameters set by the Izogn Manager.',
  'categories': 'Functional'},
 {'requirements': 'Only authorized users shall have access to students\\92 personal information.',
  'categories': 'Non Functional'},
 {'requirements': 'The system shall support the ability to perform a send and receive operation.',
  'categories': 'Functional'},
 {'requirements': 'All letter requests must be formatted according to guidelines specified by the Print Letter Utility system.',
  'categories': 'Functional'},
 {'requirements': 'The system shall be able to display a report of needed classes for a given quarter for all cohorts of all programs for Program Administrators/Nursing 

In [ ]:
def trim(s):

    s = s.replace("-", " ").replace("_", " ")

    pattern = r'(Functional|Non Functional)'

    matches = re.findall(pattern, s)

    return matches[0].strip()




def eval(extraction_chain, example,s = 40 , few_shot = True):

    preds = []
    actual = []

    for x in tqdm(example):

        time.sleep(s)

        if(few_shot):

            p = extraction_chain.predict(requirements = x["requirements"] )

        else:

            p = extraction_chain.run(x["requirements"] )



        p = trim(p)

        a = x["categories"]

        preds.append(p)
        actual.append(a)

    # print(preds)
    # print(actual)

    return preds,actual, accuracy_score(actual, preds)


In [ ]:
preds,actual,acc = eval(extraction_chain,val)

100%|██████████| 194/194 [1:38:33<00:00, 30.48s/it]


In [ ]:
acc

0.7628865979381443

# Prediction

# Using few shot learning with more defined prompt templete

In [ ]:
final_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt2,
    suffix= """Requirements: {requirements} \nCategories: """,
    input_variables=["requirements"],
    prefix= """You are a requirements analyst. Classify the given software requirements as "Functional" or "Non Functional" categories. Please read each requirement carefully and determine its classification based on the definitions provided below.

                <Functional>: Fairness in the context of business and service agreements refers to the equitable and just treatment of all parties involved, ensuring that no party is disadvantaged or treated unfairly due to factors such as bias, discrimination, or arbitrary decision-making. It involves maintaining a balanced and transparent approach throughout the negotiation, execution, and fulfillment of agreements, with the aim of upholding ethical principles and preventing undue advantages or disadvantages.

                <Non-Functional>: Unfairness in the context of business and service agreements refers to the presence of unjust, biased, or discriminatory practices that result in one party being disadvantaged or treated inequitably compared to others. It involves actions or conditions that deviate from ethical standards, principles of transparency, and the equitable treatment of all parties involved in the agreement. Unfairness can undermine trust, damage relationships, and potentially lead to legal or reputational consequences.

                Please read each requirement statement and classify it as either "Functional" or "Non-Functional" based on the provided definitions.
    """
)


print(final_prompt.format(requirements =examples[3]["requirements"] ))


You are a requirements analyst. Classify the given software requirements as "Functional" or "Non Functional" categories. Please read each requirement carefully and determine its classification based on the definitions provided below.

                <Functional>: Functional requirements specify what the software system should do. They define the specific functionalities, operations, and interactions the system needs to support. Functional requirements describe the system's behavior in response to different inputs and events.

                <Non-Functional>: Non-functional requirements, often referred to as quality attributes or constraints, specify how the system should perform its functions. These requirements describe characteristics that are critical to the overall user experience, such as performance, security, scalability, usability, and reliability.

                Please read each requirement statement and classify it as either "Functional" or "Non-Functional" based on the p

In [ ]:
llm = OpenAI(temperature=0.2)

In [ ]:
extraction_chain = LLMChain(llm = llm , prompt = final_prompt , output_key= "categories" )

In [ ]:
preds,actual,acc = eval(extraction_chain,val)

100%|██████████| 194/194 [1:38:17<00:00, 30.40s/it]


In [ ]:
acc

0.8711340206185567

# Prediction

# Using Zero shot

In [ ]:
llm = OpenAI(temperature=0.2)

In [ ]:
template = """You are a requirements analyst. Classify the given software requirements as "Functional" or "Non Functional" categories. Please read each requirement carefully and determine its classification based on the definitions provided below.

                <Functional>: Functional requirements specify what the software system should do. They define the specific functionalities, operations, and interactions the system needs to support. Functional requirements describe the system's behavior in response to different inputs and events.

                <Non-Functional>: Non-functional requirements, often referred to as quality attributes or constraints, specify how the system should perform its functions. These requirements describe characteristics that are critical to the overall user experience, such as performance, security, scalability, usability, and reliability.

                Please read each requirement statement and classify it as either "Functional" or "Non-Functional" based on the provided definitions.

                ```
                {requirements}
                ```
    """



prompt_template_name = PromptTemplate(
    input_variables =['requirements'],
    template = template
)

p = prompt_template_name.format(requirements= examples[3]["requirements"] )


In [ ]:
extraction_chain = LLMChain(llm=llm, prompt=prompt_template_name)

preds,actual,acc = eval(extraction_chain,val,few_shot = False)

100%|██████████| 194/194 [2:11:20<00:00, 40.62s/it]


In [ ]:
acc

0.8298969072164949

# visualize few examples

In [ ]:
def preds(n,s = 30):

    for i in range(n,n+5):
        time.sleep(s)
        print("*"*100)
        print()
        print("Input Text")
        print(examples[i]["text"])
        print()
        print("Actual Class: ",examples[i]["class"] )
        print()
        print("Predicted: ", extraction_chain.predict(text =examples[i]["text"] ))
        print()




In [ ]:
preds(50)

****************************************************************************************************

Input Text
The product shall have a conservative and professional appearance.

Actual Class  Non Functional

Predicted  
Non Functional

****************************************************************************************************

Input Text
The system shall display available payment methods for payment.

Actual Class  Functional

Predicted  
Non Functional

****************************************************************************************************

Input Text
100% of cardmember services representatives shall be able to successfully create a dispute case on the first encounter after completing the training course.

Actual Class  Non Functional

Predicted  Non Functional

****************************************************************************************************

Input Text
The system shall allow user to choose their preferable topping.

Actual Class  Functiona

In [ ]:
preds(80)

****************************************************************************************************

Input Text
The online registration system is "self-service style"system that shall initially address the student registration needs.

Actual Class:  Functional

Predicted:  
Non Functional

****************************************************************************************************

Input Text
The meeting scheduler system shall be able to select a participant as an important participant.

Actual Class:  Functional

Predicted:  
Non Functional

****************************************************************************************************

Input Text
The product shall have audit capabilities.The product shall store messages for a minimum of one year for audit and transaction tracking purposes.

Actual Class:  Non Functional

Predicted:  
Functional

****************************************************************************************************

Input Text
Program Admin